In [1]:
%pip install --quiet "evadb[document,notebook]"

Note: you may need to restart the kernel to use updated packages.


In [2]:
import evadb
cursor = evadb.connect().cursor()

In [3]:
%pip install requests

Note: you may need to restart the kernel to use updated packages.


In [23]:
import requests


def get_lat_lng_from_address(address, api_key):
    endpoint_url = "https://maps.googleapis.com/maps/api/geocode/json"

    params = {
        'address': address,
        'key': api_key
    }

    response = requests.get(endpoint_url, params=params)
    result = response.json()

    if 'results' in result and len(result['results']) > 0:
        location = result['results'][0]['geometry']['location']
        return f"{location['lat']},{location['lng']}"
    else:
        return None

def get_nearby_restaurants(address, api_key):
    location = get_lat_lng_from_address(address, api_key)
    if not location:
        return []

    endpoint_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # radius calculated in meters
    params = {
        'location': location,
        'radius': 500,
        'type': 'restaurant',
        'key': api_key
    }

    response = requests.get(endpoint_url, params=params)
    result = response.json()

    if 'results' in result:
        restaurants = [place for place in result['results']]
        return restaurants
    else:
        return []
    
def get_place_details(place_id, api_key):
    endpoint_url = "https://maps.googleapis.com/maps/api/place/details/json"
    params = {
        'place_id': place_id,
        'key': api_key,
        'fields': 'rating,user_ratings_total,reviews'
    }
    response = requests.get(endpoint_url, params=params)
    return response.json().get('result', {})

In [5]:
def remove_chars_from_string(data):
    data_str = str(data)
    cleaned_str = data_str.replace("'", "").replace("{", "").replace("}", "").replace("\"", "")
    return cleaned_str
    

In [34]:
api_key = ""
address = "Emory Point"
restaurant_list = []
restaurant_names = []
restaurants = get_nearby_restaurants(address, api_key)
# print(restaurants)
for index, restaurant in enumerate(restaurants, 1):
    restaurant_list.append({'name': restaurant['name'], 'price_level': restaurant.get('price_level'), 'user_ratings_total':restaurant.get('user_ratings_total'), 'wifi': restaurant.get('wifi'), 'pets': restaurant.get('pets')})
    restaurant_names.append(restaurant['name'])
print(restaurant_list)
restaurant_list = remove_chars_from_string(restaurant_list)
print(restaurant_list)

[{'name': 'The General Muir', 'price_level': 2, 'user_ratings_total': 1867, 'wifi': None, 'pets': None}, {'name': 'BORUBORU', 'price_level': 2, 'user_ratings_total': 380, 'wifi': None, 'pets': None}, {'name': 'The Gumbo Gourmet', 'price_level': 1, 'user_ratings_total': 15, 'wifi': None, 'pets': None}, {'name': 'Desta Ethiopian Kitchen Emory Point', 'price_level': 2, 'user_ratings_total': 118, 'wifi': None, 'pets': None}, {'name': 'Srithai: Thai Kitchen & Sushi Bar (Emory)', 'price_level': 2, 'user_ratings_total': 143, 'wifi': None, 'pets': None}, {'name': "Chick'nCone, Emory Point, Atlanta", 'price_level': 1, 'user_ratings_total': 107, 'wifi': None, 'pets': None}, {'name': 'The Point', 'price_level': None, 'user_ratings_total': 21, 'wifi': None, 'pets': None}]
[name: The General Muir, price_level: 2, user_ratings_total: 1867, wifi: None, pets: None, name: BORUBORU, price_level: 2, user_ratings_total: 380, wifi: None, pets: None, name: The Gumbo Gourmet, price_level: 1, user_ratings_tot

In [7]:
!apt -qq install postgresql
!service postgresql start

'apt' is not recognized as an internal or external command,
operable program or batch file.
'service' is not recognized as an internal or external command,
operable program or batch file.


In [30]:
!pip install psycopg2
import psycopg2


params = {
    "user": "postgres",
    "password": "",
    "host": "localhost",
    "port": "5432",
    "database": "evaDB_project",
}

# query = f"DROP DATABASE postgres_data;"
# cursor.query(query).df()

query = f"CREATE DATABASE postgres_data WITH ENGINE = 'postgres', PARAMETERS = {params};"
cursor.query(query).df()

,0
0,The database postgres_data has been successful...


In [35]:
user_preference = cursor.query("""
  USE postgres_data {
    select * from user_preference
  }
""").df()

In [36]:
user_preference

,user_name,price_preference,user_id,cuisine_preferences,dietary_preferences,current_cravings,other_preferences
0,Alice,3,1,Italian,"Vegan, Gluten-Free",Pasta Alfredo,pets
1,Bob,2,2,Chinese,Gluten-Free,"Dim Sum, Ice-cream",wifi
2,Charlie,3,3,Indian,Vegetarian,Paneer Tikka,outdoor seating


In [ ]:
# Set your OpenAI key as an environment variable
import os
os.environ['OPENAI_KEY'] = 'sk-'
# open_ai_key = os.environ.get('OPENAI_KEY')

In [ ]:
chatgpt_udf = f"""
    SELECT ChatGPT("Given the list of restaurants, which one would you recommend, only give one recomendation:  {restaurant_list}, the number supplied means the user'' s price preference, these are the user''s name and their respective preferences, please put all information into consideration for this user: " , user_name, cuisine_preferences, dietary_preferences, current_cravings, price_preference) 
    FROM postgres_data.user_preference;
"""
print(type(chatgpt_udf))
result = cursor.query(chatgpt_udf).df()

In [ ]:
print(result['chatgpt.response'][0])
print(result['chatgpt.response'][1])
print(result['chatgpt.response'][2])

In [ ]:
def recommend_restaurants(restaurants, user_preferences, strong_preferences):
    # Filter out restaurants not meeting strong preferences
    for preference in strong_preferences:
        restaurants = restaurants[restaurants['amenities'].apply(lambda x: preference in x)]

    # Scoring mechanism could be adjusted
    restaurants['score'] = 0
    for index, restaurant in restaurants.iterrows():
        if user_preferences['cuisine_preferences'] in restaurant['cuisine']:
            restaurants.at[index, 'score'] += 2
        if user_preferences['price_preference'] >= restaurant['price_level']:
            restaurants.at[index, 'score'] += 1
        for preference in user_preferences['other_preferences']:
            if preference in restaurant['amenities']:
                restaurants.at[index, 'score'] += 1

    # Sort based on score
    recommended = restaurants.sort_values(by='score', ascending=False)

    return recommended.head(1)  # Return top recommendation




In [ ]:
user_preferences = {
    'cuisine_preferences': 'Italian',
    'price_preference': 3,
    'other_preferences': ['wifi', 'parking']
}

strong_preferences = ['wifi', 'pet-friendly']  # Strong preferences
recommended_restaurant = recommend_restaurants(restaurants_df, user_preferences, strong_preferences)

In [ ]:
import googlemaps

def calculate_route(user_location, destination, api_key):
    gmaps = googlemaps.Client(key=api_key)
    directions = gmaps.directions(user_location, destination, mode="driving")

    if directions:
        route = directions[0]['legs'][0]
        return {
            "distance": route['distance']['text'],
            "duration": route['duration']['text'],
            "start_address": route['start_address'],
            "end_address": route['end_address'],
            "steps": [step['html_instructions'] for step in route['steps']]
        }
    else:
        return None


In [ ]:
user_location = input("Please enter your current location (address or coordinates): ")
recommended_restaurant = recommend_restaurants(restaurants_df, user_preferences, strong_preferences)
if recommended_restaurant:
    restaurant_address = recommended_restaurant['address']

    # Calculate route from user location to the recommended restaurant
    route_info = calculate_route(user_location, restaurant_address, api_key)
    if route_info:
        print(f"Distance: {route_info['distance']}")
        print(f"Duration: {route_info['duration']}")
        print("Directions:")
        for step in route_info['steps']:
            print(step)
    else:
        print("No route found.")
else:
    print("No restaurant recommendation found.")

